In [147]:
from dataclasses import dataclass, field
from typing import *
import rsa
import hashlib
import time

@dataclass
class Transaction:
    sender_address: rsa.PublicKey
    recipient_address: rsa.PublicKey
    value: float
    signature: bytes = field(default=b'')

    def sign(self, sender_private_key: rsa.PrivateKey) -> None:
        """Takes a private key, which the sender should never share, and signs this transaction to verify
        they want to transfer the tokens. We sign the core attributes of the transaction using the private
        key. We will use SHA-256 as the signature hashing algorithm."""
        self.signature = rsa.sign(self.get_core_data(), sender_private_key, 'SHA-256')
        self.validate()

    def get_core_data(self) -> bytes:
        """Provides the core info needed to sign the transaction--basically everything BUT the
        signature."""
        return (str(self.sender_address) + str(self.recipient_address) + str(self.value)).encode()

    def validate(self) -> None:
        try:
            rsa.verify(self.get_core_data(), self.signature, self.sender_address)
        except Exception as e:
            self.validation_failure_reason = str(e)
            raise

    def get_validation_failure_reason(self) -> Optional[str]:
        return self.validation_failure_reason
@dataclass
class Block:
    num: int
    timestamp: float
    prev_block_hash: str
    transactions: List[Transaction]
    block_hash: str = field(default="")
    nonce: int = field(default=0)

    def hash(self) -> str:
        """Uses SHA-256 to hash the header of the block (the core attributes of the block).
        Saves the hash to the block object for later use and also returns the hash in hex format."""
        self.block_hash = hashlib.sha256(self.get_header()).hexdigest()
        return self.block_hash

    def get_header(self) -> bytes:
        """Returns a string that represents the core attributes that uniquely identify the block AND link
        it to the previous block (forming the chain). These attributes include: block num, timestamp,
        previous block hash, transactions (real chains use a Merkle root hash for efficiency), and a nonce."""
        return (str(self.num) + str(self.timestamp) + str(self.prev_block_hash) + str(self.nonce) + str(self.transactions)).encode()

    def validate(self, proof_of_work_func: Callable[[str], bool]) -> bool:
        """Using the given function, ensure that this block hashes correctly, adhering to the agreed-upon
        consensus algorithm."""
        return proof_of_work_func(self.hash())

REWARD_AMOUNT = 2.0

class BlockchainNode:

    def __init__(self, miner_address) -> None:
        self.miner_address: rsa.PublicKey = miner_address
        self.blocks: List[Block] = []
        self.pending_transactions: List[Transaction] = []
        self.proof_of_work_func: Callable[[str], bool] = lambda x: x.endswith('000')

        self.mine_block()
    def submit_transaction(self, transaction: Transaction) -> None:
        """This is used to submit a new transaction to the chain. End-users send transactions and aren't
        concerned about the blocks, per se. This function takes a signed transaction will validate it is
        cryptographically sound. It will then need to check that there is sufficient balance to make the
        transfer. If these are good, we will add the transaction to a list of those that will be considered
        when the next block is mined."""

        try:
            # Ensure the transaction is properly signed by the private key
            transaction.validate()

            # Make sure the funds exist for the requested transaction
            sender_balance = self.get_balance(transaction.sender_address)

            if sender_balance < transaction.value:
                raise Exception(f"Insufficient tokens available ({transaction.value} required, {sender_balance} available)")

            # Transaction checks out--add to the list of pending transactions!
            self.pending_transactions.append(transaction)
            print("Transaction submitted successfully.")

        except Exception as e:
            print(f"Transaction submission failed: {e}")

            # Print additional information for better understanding of the failure
            print(f"Transaction Details:")
            print(f"Sender Public Key: {transaction.sender_address}")
            print(f"Recipient Public Key: {transaction.recipient_address}")
            print(f"Value: {transaction.value}")
            print(f"Signature: {transaction.signature}")

            # Check if there is a validation failure reason
            validation_failure_reason = transaction.get_validation_failure_reason()
            if validation_failure_reason:
                print(f"Validation Failure Reason: {validation_failure_reason}")

            # Optionally, re-raise the exception to halt program execution if needed
            raise



    def mine_block(self) -> None:
        """This function bundles all pending transactions and mines a new block. Mining is the process
        by which a node creates a new block. This is where the decentralized consensus algorithm comes in.
        We will be using a proof-of-work algorithm that is computationally expensive. This prevents bad actor
        nodes from flooding the P2P network with invalid transactions, blocks, or chains. It is too expensive to
        rewrite history, making the blockchain more secure from these kinds of attacks. This function will mine
        the new block and then append it to the chain automatically.

        Nodes that successfully mine a new block get a token award. This is how new tokens show up in the chain
        and this is supposed to incentivize more peers on the network.
        """

        # Make sure we have a genesis block to start the chain
        if len(self.blocks) <= 0:
            prev_hash = "-"
            num = 0
        else:
            prev_block = self.blocks[-1]
            prev_hash = prev_block.hash()
            num = prev_block.num + 1

        new_block = Block(
            num=num,
            timestamp=time.time(),
            prev_block_hash=prev_hash,
            transactions=self.pending_transactions
        )

        proof = 0
        while not new_block.validate(self.proof_of_work_func):
            proof += 1
            new_block.nonce = proof

        self.blocks.append(new_block)
        self.pending_transactions = []  # reset the list of pending transactions

        # Reward the miner
        reward_transaction = Transaction(
            sender_address=rsa.PublicKey(0, 0),  # 0 address indicates a reward
            recipient_address=self.miner_address,
            value=REWARD_AMOUNT
        )

        self.pending_transactions.append(reward_transaction)
        print(f"Successfully mined new block {new_block.num}!")

    def validate_chain(self) -> bool:
        """This function validates the integrity of the entire blockchain. It will check the
        hash of each block and make sure that they are consistent with the proof-of-work
        consensus algorithm. Additionally, it will check that each block correctly points to the
        previous block (by including the hash of the previous block). If any part of the blockchain
        is invalid, this function returns False. Otherwise, it returns True."""
        for i in range(1, len(self.blocks)):
            prev_block = self.blocks[i - 1]
            current_block = self.blocks[i]

            # Check if the previous block hash matches
            if current_block.prev_block_hash != prev_block.hash():
                print(f"Blockchain validation failed at block {current_block.num}: Previous block hash mismatch.")
                return False

            # Check if the current block hash is valid according to the proof-of-work function
            if not current_block.validate(self.proof_of_work_func):
                print(f"Blockchain validation failed at block {current_block.num}: Invalid proof-of-work.")
                return False

        print("Successfully validated chain of size {}!".format(len(self.blocks)))
        return True

    def get_balance(self, address: rsa.PublicKey) -> float:
        """Calculates the current balance of the given address by iterating through
        all the blocks and transactions in the blockchain."""
        balance = 0.0

        for block in self.blocks:
            for transaction in block.transactions:
                if transaction.sender_address == address:
                    balance -= transaction.value
                if transaction.recipient_address == address:
                    balance += transaction.value

        return balance

In [148]:
# Key Generation
import rsa

# Blockchain Node Key Pair
public_key, private_key = rsa.newkeys(512)
print("Blockchain Node Public Key:", public_key)
print("Blockchain Node Private Key:", private_key)

# Receiving Address Key Pair
receiving_public_key, receiving_private_key = rsa.newkeys(512)
print("\nReceiving Address Public Key:", receiving_public_key)
print("Receiving Address Private Key:", receiving_private_key)


Blockchain Node Public Key: PublicKey(8033280519126748152862541805898902269863348704797156188637998775386139881358258929999785158108508023602693802533646486324616880302029305579740015900095291, 65537)
Blockchain Node Private Key: PrivateKey(8033280519126748152862541805898902269863348704797156188637998775386139881358258929999785158108508023602693802533646486324616880302029305579740015900095291, 65537, 6315741928193321308211277088498724986720616476412919001778733950314644506879878105354867806621052804729635542924824178549631725716130883307908152564477273, 6590931002713174277155744953758248758560548879865727915062687586548247844181757509, 1218838509433618234064413514100915128252053168863639250339587725437539199)

Receiving Address Public Key: PublicKey(9220855083508811264844086483653192883836863027394105807409765430804128650054401752397092005660051437380278495521898156768158047925560458029993839481113389, 65537)
Receiving Address Private Key: PrivateKey(92208550835088112648440864836531928

In [149]:
# Blockchain Node Initialization
node = BlockchainNode(public_key)

# Transaction Creation and Signing
t1 = Transaction(public_key, receiving_public_key, 0.5)
t1.sign(private_key)
print("\nTransaction 1:")
print("Sender Public Key:", t1.sender_address)
print("Recipient Public Key:", t1.recipient_address)
print("Value:", t1.value)
print("Signature:", t1.signature)


Successfully mined new block 0!

Transaction 1:
Sender Public Key: PublicKey(8033280519126748152862541805898902269863348704797156188637998775386139881358258929999785158108508023602693802533646486324616880302029305579740015900095291, 65537)
Recipient Public Key: PublicKey(9220855083508811264844086483653192883836863027394105807409765430804128650054401752397092005660051437380278495521898156768158047925560458029993839481113389, 65537)
Value: 0.5
Signature: b"I\xbcQ:\xa8}\xbd\xeb\x8d\x85\x96f\x8fE\xbaE\xe3t\xf4BX8\x83\xa1\xc2\xecJ\xe7\x062\x99\xf0|6\x92k\\K\xd9\xaa\xd1\x9b\xedT\xde\xd5']\xb52b\xda\xff\xa1\x9a4Rd\x99\n\xe6\xf8_%"


In [150]:
# Mine the First Block
node.mine_block()

# Display Block Information
first_block = node.blocks[0]
print("\nFirst Block Information:")
print("Block Number:", first_block.num)
print("Timestamp:", first_block.timestamp)
print("Previous Block Hash:", first_block.prev_block_hash)
print("Transactions:", first_block.transactions)
print("Block Hash:", first_block.block_hash)
print("Nonce:", first_block.nonce)


Successfully mined new block 1!

First Block Information:
Block Number: 0
Timestamp: 1703089267.2591033
Previous Block Hash: -
Transactions: []
Block Hash: 1c067be1fedb1b9f17656df82ab25af0f16a88893b4c3ebdc0ff1651f7460000
Nonce: 269


In [151]:
# Submitting Another Transaction
t2 = Transaction(public_key, receiving_public_key, 1.0)
t2.sign(private_key)
node.submit_transaction(t2)

# Mine the Second Block
node.mine_block()

# Display Updated Balances
print("\nUpdated Balances:")
print("Blockchain Node Balance:", node.get_balance(public_key))
print("Receiving Address Balance:", node.get_balance(receiving_public_key))


Transaction submitted successfully.
Successfully mined new block 2!

Updated Balances:
Blockchain Node Balance: 3.0
Receiving Address Balance: 1.0


In [152]:
# Validate the Blockchain
node.validate_chain()


Successfully validated chain of size 3!


True

In [154]:
# Transaction 4 (intentional signature modification)
t4 = Transaction(public_key, receiving_public_key, 0.8)
t4.sign(private_key)
t4.signature = b'invalid_signature'

try:
    node.submit_transaction(t4)
    print("Transaction 4 submitted successfully.")
except Exception as e:
    print(f"t4 failed")

# Uncomment the following lines based on your testing scenario
# node.mine_block()
# node.validate_chain()


Transaction submission failed: Verification failed
Transaction Details:
Sender Public Key: PublicKey(8033280519126748152862541805898902269863348704797156188637998775386139881358258929999785158108508023602693802533646486324616880302029305579740015900095291, 65537)
Recipient Public Key: PublicKey(9220855083508811264844086483653192883836863027394105807409765430804128650054401752397092005660051437380278495521898156768158047925560458029993839481113389, 65537)
Value: 0.8
Signature: b'invalid_signature'
Validation Failure Reason: Verification failed
t4 failed


In [155]:
# Transaction 5 (intentional recipient key modification)
t5 = Transaction(public_key, receiving_public_key, 1.2)
t5.sign(private_key)

# Create a new transaction with a different recipient key to simulate a verification failure
t5_modified = Transaction(public_key, rsa.newkeys(512)[0], 1.2)
t5_modified.signature = t5.signature  # Use the signature from the original transaction

try:
    node.submit_transaction(t5_modified)
    print("Transaction submitted successfully.")
except Exception as e:
    print(f"t5 failed")


Transaction submission failed: Verification failed
Transaction Details:
Sender Public Key: PublicKey(8033280519126748152862541805898902269863348704797156188637998775386139881358258929999785158108508023602693802533646486324616880302029305579740015900095291, 65537)
Recipient Public Key: PublicKey(6858075679425024721643534758096336486517575415116396797260703168482205587016441234654512574920167830522413094579043512951733064009843802124441563916905689, 65537)
Value: 1.2
Signature: b'\x8b[\xb0\x1a\xd8L\x89\x0f\xfb7\x0e\x0c\x9f)\x90m.%\x07\xb9Ryl\xf0"\x04nQ\xacb\x80\xd5\xc2\xcaQy\x10\x04\x11\xd3t]&\x82\xae\xa1\xddJ\x90q*>D\xf5Cb\xd3\x8c\xa0Q\xc0\xeeT\xb7'
Validation Failure Reason: Verification failed
t5 failed
